# Часть 1. SQL запросы:

Запросы протестированы на тестовых таблицах (если нажать Run, выведутся таблицы в Results):

https://www.db-fiddle.com/f/r9FdZhDADUaUKsoWAZJ64Q/4

#### 1. Для каждой страны выведите самый часто покупаемый оффер за последний месяц и его содержимое. 

1) Объединила 3 таблицы и добавила фильтр, чтобы остались только транзакции за послений месяц (текущая дата минус интервал в один месяц);

2) Далее с помощью оконной функции проранжировала offer_id (1 - наиболее часто встречающееся значение) в группах country (поздапрос rank_subquery);

3) В рузультирующую таблицу вывела страны и наиболее часто покупаемые офферы в них. Для читаемости помимо содержимого оставила id оффера и его называние.

In [ ]:
SELECT 
    country, 
    offer_id, 
    name, 
    content
FROM (
    SELECT 
        country,
        offer_id,
        name,
        content,
        COUNT(offer_id) AS count_offer_id,
        ROW_NUMBER() OVER (PARTITION BY country ORDER BY COUNT(offer_id) DESC) AS rang
    FROM
        (        
        SELECT                                                         
            a.date_index AS date_index,
            a.player_id AS player_id,
            a.offer_id AS offer_id,
            a.price AS price,
            b.level AS level,
            b.country AS country,
            b.date_install AS date_install,
            c.name,
            c.content
        FROM TRANSACTIONS AS a
        LEFT JOIN PLAYER_INFO AS b
            ON a.player_id = b.player_id
        LEFT JOIN OFFERS AS c
            ON a.offer_id = c.offer_id
        WHERE date_index BETWEEN date_trunc('day', NOW()) - interval '1 month' AND date_trunc('day', NOW())) AS subq
    GROUP BY country, offer_id, name, content
    ) AS rank_subquery
WHERE rang = 1
ORDER BY country

#### 2. Для игроков разных уровневых групп (0-10, 10-20, 20+) выведите средний чек, минимальную и максимальную сумму транзакции.

1) Объединила 3 исходные таблицы;

2) Добавила условие для дальнейшей группировки по интервалам уровней (немного переименовала группы, чтобы 10 и 20 уровни использовались уникально);

3) Вывела данные после агрегации по группам уровней.

In [ ]:
SELECT
    CASE
    WHEN level >=0 AND level <10 THEN '0-9'
    WHEN level >=10 AND level <20 THEN '10-19'
    ELSE '20+'
    END AS level_groups,
    ROUND(AVG(price), 2) AS avg_transaction,
    MIN(price) AS min_transaction,
    MAX(price) AS max_transaction
FROM
    (
    SELECT                                                             
        a.date_index AS date_index,
        a.player_id AS player_id,
        a.offer_id AS offer_id,
        a.price AS price,
        b.level AS level,
        b.country AS country,
        b.date_install AS date_install,
        c.name,
        c.content
    FROM TRANSACTIONS AS a
    LEFT JOIN PLAYER_INFO AS b
        ON a.player_id = b.player_id
    LEFT JOIN OFFERS AS c
        ON a.offer_id = c.offer_id
    ) as subq
GROUP BY level_groups
ORDER BY level_groups

#### 3. Для каждого игрока посчитайте LTV30 (сумма покупок за 30 дней с момента установки).

In [ ]:
SELECT
    player_id,
    SUM(price) AS LTV30
FROM
    (                                                                   
    SELECT
        a.date_index AS date_index,
        a.player_id AS player_id,
        a.offer_id AS offer_id,
        a.price AS price,
        b.level AS level,
        b.country AS country,
        b.date_install AS date_install,
        c.name,
        c.content
    FROM TRANSACTIONS AS a
    LEFT JOIN PLAYER_INFO AS b
        ON a.player_id = b.player_id
    LEFT JOIN OFFERS AS c
        ON a.offer_id = c.offer_id
    ) AS subq
WHERE date_index BETWEEN date_install AND (date_install + interval '30 days')
-- WHERE date_index - date_install <= 30
GROUP BY player_id
ORDER BY player_id

Также ниже запрос на расчет показателя LTV в среднем на пользователя. Отражает, сколько в среднем приносит пользователь денег в продукт до того, как покинет его. Рассчитывается как произведение среднего дохода на пользователя (ARPU) и средней продолжительности жизни клиента (30 - данные из задания).

In [ ]:
SELECT
    ROUND(SUM(price) / COUNT (DISTINCT player_id) * 30, 2) AS LTV
FROM
    (
    SELECT
        a.date_index AS date_index,
        a.player_id AS player_id,
        a.offer_id AS offer_id,
        a.price AS price,
        b.level AS level,
        b.country AS country,
        b.date_install AS date_install,
        c.name,
        c.content
    FROM TRANSACTIONS AS a
    LEFT JOIN PLAYER_INFO AS b
        ON a.player_id = b.player_id
    LEFT JOIN OFFERS AS c
        ON a.offer_id = c.offer_id
    ) as subq